In [1]:
import re
import xlwt
import pandas as pd
import numpy as np
import requests
import urllib.error
import urllib.parse
import json
import time
pd.options.mode.chained_assignment = None  # default='warn'

In [2]:
with open('view-source_https___opshop.org_list_VIC_.html', "r", encoding="utf-8") as f:
            file = f.read()

In [3]:
store = re.findall(r'<span class="html-tag">&lt;span <span class="html-attribute-name">class</span>=\'<span class="html-attribute-value">screenBreak</span>\'&gt;</span>(.*?)id</span>=\'<span class="html-attribute-value">ID', file)

In [5]:
#file

In [6]:
list_of_name = ['null']*len(store)
list_of_address = ['null']*len(store)
list_of_time = ['null']*len(store)

In [7]:
for i in range(len(store)):
    name = re.findall(r'(.*?)<span class="html-tag">&lt;/span&gt;</span><span class="html-tag">&lt;/a&gt;</span>',store[i])
    list_of_name[i] = name[0]
    address = re.findall(r'<span class="html-tag">&lt;br /&gt;</span>(.*?)<span class="html-tag">&lt;/a&gt;</span></td></tr><tr>',store[i])
    list_of_address[i] = address[0]
    opentime = re.findall(r'col-xs-4</span>(.*?)&amp;nbsp|col-xs-4</span>(.*?)<span class="html-tag">&lt;br /&gt;</span>&amp;nbsp',store[i])
    opentime = list(opentime[0])
    opentime += ['null']
    time_detail = re.split(r'\'&gt;</span>|<span class="html-tag">&lt;br /&gt;</span>|<span class="html-tag">&lt;br/&gt;</span>|<span class="html-tag">&lt;b',opentime[0])
    list_of_time[i] = time_detail

In [8]:
for i in range(len(store)):
    for j in range(len(list_of_time[i])):
        list_of_time[i][j] = re.sub(r'amp;amp;','',list_of_time[i][j])

In [9]:
for i in range(len(store)):
    list_of_address[i] = re.sub(r'amp;amp;','',list_of_address[i])

In [10]:
def meltlist(kk):
    string = ''
    for i in range(len(kk)):
        if kk[i] == '':
            string = string
        else:
            string += kk[i]
            string += '\n'
    return string  

In [11]:
for i in range(len(list_of_time)):
    list_of_time[i] = meltlist(list_of_time[i])

In [12]:
df = pd.DataFrame(list_of_name, columns=['name'])
df = pd.concat([df, pd.DataFrame(list_of_address,columns=['address'])],axis=1)
df = pd.concat([df, pd.DataFrame(list_of_time,columns=['opentime'])],axis=1)

In [13]:
df.opentime = df.opentime.replace('','Unknown\n')

In [16]:
df['lat'] = ''
df['lng'] = ''

In [17]:
def get_latitude_longtitude(address):
    # decode url
    address = urllib.parse.quote(address)
    url = "https://maps.googleapis.com/maps/api/geocode/json?address=" + address + "&key=" #add your key    
    while True:
        res = requests.get(url)
        js = json.loads(res.text)

        if js["status"] != "OVER_QUERY_LIMIT":
            time.sleep(1)
            break

    result = js["results"][0]["geometry"]["location"]
    lat = result["lat"]
    lng = result["lng"]

    return lat, lng

In [18]:
# extra edit
df['address'][169] = '126 Canterbury Rd, Bayswater North VIC 3153'
df['address'][440] = 'Shop 9/10 Village Walk, Drysdale VIC 3222'
df['address'][468] = '178 Manifold St, Camperdown VIC 3260'
df['address'][539] = '2/8 Selkirk Ave, McKinnon VIC 3204'
df['address'][797] = '8 Ninth Ave, Rosebud VIC 3939'

In [19]:
for i in range(len(df)):
    if i >= 0:      
        address = df['address'][i]
        lat, lng = get_latitude_longtitude(address)
        df['lat'][i] = lat
        df['lng'][i] = lng


In [20]:
for i in range(len(df)):
    if df['lat'][i] > 0:
        df['address'][i] += ' VIC'

In [21]:
df[df['lat'] > 0]

,name,address,opentime,lat,lng
27,Brotherhood Deer Park,"Shop 1, 112-116 Westwood Dr, Deer Park VIC",Mon - Fri 9.30 am to 5 pm\nSat 10 am to 3 pm\n,40.4333,-84.9773
91,St Marks RECYCLED,298 Smith Street VIC,Mon - Fri 11:00am - 5:00pm\nSat 11:00am - 5.00...,40.6819,-73.9939
101,Penny Pinchers,Work For The Soul VIC,Mon - Thu 8.30 - 4.00\nFri - Sat Opening soon\...,35.6495,-97.4599
157,Red Cross,191 Bridge Road VIC,Mon - Thu 10am - 6pm\nFri 10am - 7pm\nSat 10am...,42.3442,-72.6727
371,Recycle Super store,47 Excelsior Drive VIC,Mon - Fri 9am - 5pm\nSat 9am -2pm\n,43.0748,-89.5252
445,Salvos Thrift Shop Leopold,49 Ash Road VIC,Mon - Fri 10am - 4pm\nSat 10am - 2pm\n,41.1063,-73.9887
504,Joyces Junkatique,38 Main Rd VIC,Unknown\n,41.614,-71.2571
508,Ballarat Bargain Browser,10 Eastwood St VIC,Mon - Fri 9am -4.30 pm\nSat 9am - 1pm\n,29.7486,-95.3302
510,Outreach Furniture Shop,102 Albert St VIC,Unknown\n,42.9268,-78.7008
595,Salvation Army,"Epsom Shopping Centre, EPSOM VIC",Unknown\n,51.3324,-0.268324


In [24]:
df['address'][595] = '320 Epsom Rd, Ascot Vale VIC 3032'
df['address'][829] = 'Rossiter Rd, Carrum Downs VIC 3201'

In [25]:
for i in range(len(df)):
    if i >= 0:      
        address = df['address'][i]
        lat, lng = get_latitude_longtitude(address)
        df['lat'][i] = lat
        df['lng'][i] = lng

In [27]:
##df[df['lat'] > 0]

In [28]:
df.to_csv('op_shop_list.csv')